# Pandas
[Библиотека](https://ru.wikipedia.org/wiki/Pandas) для работы с таблицами, по сути являющейся реализацией нереляционной базы данных.

Рекомендуемое ядро для запуска кода: `Python v3.7` или выше.

In [1]:
from os import mkdir
from os.path import isdir, join as join_path
from functools import partial
from warnings import filterwarnings

import numpy as np
import pandas as pd


filterwarnings('ignore')

DATA_DIR = 'class_data/'  # Папка, куда мы будем сохранять все файлы
if not isdir(DATA_DIR):
    mkdir(DATA_DIR)

to_data_dir = partial(join_path, DATA_DIR)
print(f"Пример работы функции 'to_data_dir': {to_data_dir('test.file')}")

Пример работы функции 'to_data_dir': class_data/test.file


Основным объектом для работы является `DataFrame` &mdash; таблица.

Простейший пример датафрейма:

In [2]:
df = pd.DataFrame(
    data={
        'Name': ('Andrew', 'Shaley', 'Kimberley'),
        'Sex':  ('m', 'f', 'f'),
        'Age': (22, 34, 18),
        'Wealth, $': (1e6, 1e5, -1200)
    }
)
df

,Name,Sex,Age,"Wealth, $"
0,Andrew,m,22,1000000.0
1,Shaley,f,34,100000.0
2,Kimberley,f,18,-1200.0


Давайте разберёмся, из каких элементов состоит этот объект и какой функционал он представляет.

Во-первых, это имена строк и столбцов.

Доступ к ним можно получить следующими командами:

In [3]:
df.index    # Для строк

RangeIndex(start=0, stop=3, step=1)

In [4]:
df.columns  # Для столбцов

Index(['Name', 'Sex', 'Age', 'Wealth, $'], dtype='object')

Как мы можем догадаться, структуры данных, хранящие имена строк и столбцов, называются `индексами`.

Что же это такое и зачем это нужно?

## pd.Index
### Преобразование имён строк и столбцов

В нашем примере имена строк соответствовали их порядковому номеру.

Но строки (как и столбцы) можно называть произвольным образом:

In [5]:
df.index = ['User_1', 'User_2', 'User_400']
df

,Name,Sex,Age,"Wealth, $"
User_1,Andrew,m,22,1000000.0
User_2,Shaley,f,34,100000.0
User_400,Kimberley,f,18,-1200.0


Индексы можно удобно преобразовывать:

In [6]:
df.index = df.index.map(lambda user_id: int(user_id.split('_')[-1]))
df

,Name,Sex,Age,"Wealth, $"
1,Andrew,m,22,1000000.0
2,Shaley,f,34,100000.0
400,Kimberley,f,18,-1200.0


Таким же образом названия колонок можно, например, заключить в кавычки:

In [7]:
df.columns = df.columns.map(lambda colname: f"'{colname}'")
df

,'Name','Sex','Age',"'Wealth, $'"
1,Andrew,m,22,1000000.0
2,Shaley,f,34,100000.0
400,Kimberley,f,18,-1200.0


И обратно:

In [8]:
df.columns = df.columns.map(lambda colname: colname.split("'")[1])
df

,Name,Sex,Age,"Wealth, $"
1,Andrew,m,22,1000000.0
2,Shaley,f,34,100000.0
400,Kimberley,f,18,-1200.0


### Теоретико-множественные операции
Для различных индексов можно производить простые операции, применимые к множествам. Допустим, мы имеем два датасета с разными строчными индексами, и нам нужно найти общие имена строк.

In [9]:
df_1 = pd.DataFrame(
    data={
        'Name': ('Andrew', 'Shaley', 'Kimberley'),
        'Sex':  ('m', 'f', 'f'),
        'Age': (22, 34, 18),
        'Wealth, $': (1e6, 1e5, -1200)
    },
    index=(200, 300, 400)
)
df_1

,Name,Sex,Age,"Wealth, $"
200,Andrew,m,22,1000000.0
300,Shaley,f,34,100000.0
400,Kimberley,f,18,-1200.0


In [10]:
df_2 = pd.DataFrame(
    data={
        'Name': ('Andrew', 'Shaley', 'Kimberley'),
        'Sex':  ('m', 'f', 'f'),
        'Age': (22, 34, 18),
        'Wealth, $': (1e6, 1e5, -1200)
    },
    index=(400, 300, 600)
)
df_2

,Name,Sex,Age,"Wealth, $"
400,Andrew,m,22,1000000.0
300,Shaley,f,34,100000.0
600,Kimberley,f,18,-1200.0


Сделать это можно следующим образом:

In [11]:
df_2.index.intersection(df_1.index)  # Общие имена строк: 400, 300

Int64Index([400, 300], dtype='int64')

In [12]:
df_2.index & df_1.index              # То же самое

Int64Index([400, 300], dtype='int64')

In [13]:
df_2.index.difference(df_1.index)    # Уникальные для df_2 имена строк

Int64Index([600], dtype='int64')

In [14]:
# Преобразуем имена строк обратно к виду User_ID

df.index = ['User_1', 'User_2', 'User_400']
df

,Name,Sex,Age,"Wealth, $"
User_1,Andrew,m,22,1000000.0
User_2,Shaley,f,34,100000.0
User_400,Kimberley,f,18,-1200.0


### Индексация
#### Одиночная

Можно производить выделение определённых строк из датафрейма:

In [15]:
user_1 = df.loc['User_1']
user_1

Name         Andrew
Sex               m
Age              22
Wealth, $     1e+06
Name: User_1, dtype: object

In [16]:
user_1['Wealth, $']

1000000.0

Как и выделение столбцов:

In [17]:
df['Name']

User_1         Andrew
User_2         Shaley
User_400    Kimberley
Name: Name, dtype: object

#### Множественная

Можно производить множественное выделение.

Этим свойством, например, уже не может похвастаться стандартный питоновский словарь.

In [18]:
df.loc[['User_1', 'User_400']]  # Строки

,Name,Sex,Age,"Wealth, $"
User_1,Andrew,m,22,1000000.0
User_400,Kimberley,f,18,-1200.0


In [19]:
df[['Name', 'Age']]             # Столбцы

,Name,Age
User_1,Andrew,22
User_2,Shaley,34
User_400,Kimberley,18


### Именование

Для удобства индексы можно называть своими именами

In [20]:
df.index.name = 'User ID'
df

,Name,Sex,Age,"Wealth, $"
User ID,,,,
User_1,Andrew,m,22,1000000.0
User_2,Shaley,f,34,100000.0
User_400,Kimberley,f,18,-1200.0


In [21]:
df.columns.name = 'Features'
df

Features,Name,Sex,Age,"Wealth, $"
User ID,,,,
User_1,Andrew,m,22,1000000.0
User_2,Shaley,f,34,100000.0
User_400,Kimberley,f,18,-1200.0


In [22]:
df.columns.name = None  # Таким образом можно удалить название индекса
df

,Name,Sex,Age,"Wealth, $"
User ID,,,,
User_1,Andrew,m,22,1000000.0
User_2,Shaley,f,34,100000.0
User_400,Kimberley,f,18,-1200.0


## pd.DataFrame

### Транспонирование

In [23]:
df.T

User ID,User_1,User_2,User_400
Name,Andrew,Shaley,Kimberley
Sex,m,f,f
Age,22,34,18
"Wealth, $",1e+06,100000,-1200


### Конвертация в `np.ndarray`

In [24]:
df.values

array([['Andrew', 'm', 22, 1000000.0],
       ['Shaley', 'f', 34, 100000.0],
       ['Kimberley', 'f', 18, -1200.0]], dtype=object)

### Добавление столбца

In [25]:
df['Month of birth'] = ['January', 'March', 'April']
df

,Name,Sex,Age,"Wealth, $",Month of birth
User ID,,,,,
User_1,Andrew,m,22,1000000.0,January
User_2,Shaley,f,34,100000.0,March
User_400,Kimberley,f,18,-1200.0,April


### Применение функций

In [26]:
df.apply(lambda row: row.values, axis=1)   # Построчно

User ID
User_1      [Andrew, m, 22, 1000000.0, January]
User_2         [Shaley, f, 34, 100000.0, March]
User_400     [Kimberley, f, 18, -1200.0, April]
dtype: object

In [27]:
df.apply(lambda row: set(row), axis=0)     # Поколоночно

Name                 {Shaley, Andrew, Kimberley}
Sex                                       {m, f}
Age                                 {34, 18, 22}
Wealth, $         {1000000.0, -1200.0, 100000.0}
Month of birth           {January, April, March}
dtype: object

In [28]:
df.applymap(lambda x: str(x) + ' Some Suffix')  # Поэлементно

,Name,Sex,Age,"Wealth, $",Month of birth
User ID,,,,,
User_1,Andrew Some Suffix,m Some Suffix,22 Some Suffix,1000000.0 Some Suffix,January Some Suffix
User_2,Shaley Some Suffix,f Some Suffix,34 Some Suffix,100000.0 Some Suffix,March Some Suffix
User_400,Kimberley Some Suffix,f Some Suffix,18 Some Suffix,-1200.0 Some Suffix,April Some Suffix


### Конкатенация

In [29]:
pd.concat((df, df))

,Name,Sex,Age,"Wealth, $",Month of birth
User ID,,,,,
User_1,Andrew,m,22,1000000.0,January
User_2,Shaley,f,34,100000.0,March
User_400,Kimberley,f,18,-1200.0,April
User_1,Andrew,m,22,1000000.0,January
User_2,Shaley,f,34,100000.0,March
User_400,Kimberley,f,18,-1200.0,April


In [30]:
pd.concat((df, df), axis=1)

,Name,Sex,Age,"Wealth, $",Month of birth,Name,Sex,Age,"Wealth, $",Month of birth
User ID,,,,,,,,,,
User_1,Andrew,m,22,1000000.0,January,Andrew,m,22,1000000.0,January
User_2,Shaley,f,34,100000.0,March,Shaley,f,34,100000.0,March
User_400,Kimberley,f,18,-1200.0,April,Kimberley,f,18,-1200.0,April


### Подсчёт статистик

`pd.DataFrame` имеет множество методов для расчёта различных статистик.

Как пример, можно посчитать среднее значение.

In [31]:
df = pd.DataFrame(
    [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]],
    index=[100, 200, 300]
)
df

,0,1,2
100,1,2,3
200,4,5,6
300,7,8,9


In [32]:
df.mean()         # По столбцам

0    4.0
1    5.0
2    6.0
dtype: float64

In [33]:
df.mean(axis=1)   # По строкам

100    2.0
200    5.0
300    8.0
dtype: float64

In [34]:
df.mean().mean()  # По всем элементам

5.0

Но лучше использовать метод, приведённый ниже

In [35]:
df.values.mean()

5.0

### Операции
Почти то же самое, что и в `NumPy`.

In [36]:
np.exp(df)

,0,1,2
100,2.718282,7.389056,20.085537
200,54.598150,148.413159,403.428793
300,1096.633158,2980.957987,8103.083928


In [37]:
df + df

,0,1,2
100,2,4,6
200,8,10,12
300,14,16,18


### Другие полезные методы

In [38]:
df.info()     # Summary по потреблению памяти и хранящихся типах данных

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 100 to 300
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       3 non-null      int64
 1   1       3 non-null      int64
 2   2       3 non-null      int64
dtypes: int64(3)
memory usage: 96.0 bytes


In [39]:
df.head(2)    # Первые n строк

,0,1,2
100,1,2,3
200,4,5,6


In [40]:
df.tail(2)    # Последние n строк

,0,1,2
200,4,5,6
300,7,8,9


In [41]:
df.sample(2)   # Рандомные n строк

,0,1,2
300,7,8,9
200,4,5,6


In [42]:
df.describe()  # Различные статистики

,0,1,2
count,3.0,3.0,3.0
mean,4.0,5.0,6.0
std,3.0,3.0,3.0
min,1.0,2.0,3.0
25%,2.5,3.5,4.5
50%,4.0,5.0,6.0
75%,5.5,6.5,7.5
max,7.0,8.0,9.0


### Сохранение в текстовый файл

In [43]:
df.to_csv(to_data_dir('df.tsv'), sep='\t')

In [44]:
with open(to_data_dir('df.tsv'), 'r') as out:
    print(''.join(out.readlines()))

	0	1	2
100	1	2	3
200	4	5	6
300	7	8	9



### Чтение из текстового файла

In [45]:
df = pd.read_csv(to_data_dir('df.tsv'), sep='\t', index_col=0)
df

,0,1,2
100,1,2,3
200,4,5,6
300,7,8,9


### Сохранение в бинарный формат
Преимущества:
- Значительно быстрее для чтения и записи на диск
- Обычно меньше по размеру, чем аналогичный текстовый файл

Оказываются существенными при работе с файлами крупнее десятков мегабайт.

Минус:
- Нечитаем для человека

In [46]:
df.to_pickle(to_data_dir('df.pkl'))

### Чтение бинарного формата

In [47]:
df = pd.read_pickle(to_data_dir('df.pkl'))

In [48]:
df

,0,1,2
100,1,2,3
200,4,5,6
300,7,8,9


Давайте теперь разберёмся, какая структура данных возвращается при одиночном выделении столбцов или строк

In [49]:
df = pd.DataFrame(
    data={
        'Name': ('Andrew', 'Shaley', 'Kimberley'),
        'Sex':  ('m', 'f', 'f'),
        'Age': (22, 34, 18),
        'Wealth, $': (1e6, 1e5, -1200)
    }
)
df

,Name,Sex,Age,"Wealth, $"
0,Andrew,m,22,1000000.0
1,Shaley,f,34,100000.0
2,Kimberley,f,18,-1200.0


In [50]:
df['Sex']

0    m
1    f
2    f
Name: Sex, dtype: object

`df['Sex']` имеет тип `pd.Series`. Что же это такое?

## pd.Series

Это одномерный набор данных, аналог питоновского словаря, но с поддержкой множественной индексации и огромным числом удобных методов для подсчета статистик, сортировки и прочего.

Как и `pd.DataFrame`, имеет индекс, то есть набор ключей словаря.

In [51]:
series = pd.Series(
    index=map(lambda x: f'User_ID {x}', range(100)),
    data=np.random.choice(('m', 'f'), 100, p=(0.6, 0.4)),
    name='Sex'
)
series

User_ID 0     f
User_ID 1     m
User_ID 2     m
User_ID 3     f
User_ID 4     m
             ..
User_ID 95    m
User_ID 96    m
User_ID 97    m
User_ID 98    f
User_ID 99    f
Name: Sex, Length: 100, dtype: object

In [52]:
series.index

Index(['User_ID 0', 'User_ID 1', 'User_ID 2', 'User_ID 3', 'User_ID 4',
       'User_ID 5', 'User_ID 6', 'User_ID 7', 'User_ID 8', 'User_ID 9',
       'User_ID 10', 'User_ID 11', 'User_ID 12', 'User_ID 13', 'User_ID 14',
       'User_ID 15', 'User_ID 16', 'User_ID 17', 'User_ID 18', 'User_ID 19',
       'User_ID 20', 'User_ID 21', 'User_ID 22', 'User_ID 23', 'User_ID 24',
       'User_ID 25', 'User_ID 26', 'User_ID 27', 'User_ID 28', 'User_ID 29',
       'User_ID 30', 'User_ID 31', 'User_ID 32', 'User_ID 33', 'User_ID 34',
       'User_ID 35', 'User_ID 36', 'User_ID 37', 'User_ID 38', 'User_ID 39',
       'User_ID 40', 'User_ID 41', 'User_ID 42', 'User_ID 43', 'User_ID 44',
       'User_ID 45', 'User_ID 46', 'User_ID 47', 'User_ID 48', 'User_ID 49',
       'User_ID 50', 'User_ID 51', 'User_ID 52', 'User_ID 53', 'User_ID 54',
       'User_ID 55', 'User_ID 56', 'User_ID 57', 'User_ID 58', 'User_ID 59',
       'User_ID 60', 'User_ID 61', 'User_ID 62', 'User_ID 63', 'User_ID 64',
       'U

In [53]:
series.values

array(['f', 'm', 'm', 'f', 'm', 'm', 'm', 'm', 'f', 'f', 'm', 'f', 'f',
       'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'm', 'f', 'f',
       'f', 'm', 'm', 'm', 'm', 'f', 'm', 'm', 'f', 'f', 'f', 'f', 'f',
       'm', 'f', 'f', 'f', 'f', 'f', 'm', 'm', 'm', 'f', 'f', 'm', 'f',
       'f', 'f', 'm', 'm', 'f', 'f', 'm', 'm', 'f', 'f', 'f', 'f', 'm',
       'f', 'm', 'm', 'm', 'm', 'f', 'm', 'f', 'm', 'm', 'm', 'f', 'm',
       'f', 'f', 'f', 'm', 'f', 'm', 'm', 'm', 'm', 'f', 'f', 'm', 'm',
       'm', 'm', 'f', 'f', 'm', 'm', 'm', 'f', 'f'], dtype=object)

In [54]:
series = pd.Series(
    index=map(lambda x: f'User_ID {x}', range(100)),
    data=np.random.choice([0, 1, 2, 3], 100),
    name='Some values'
)
series

User_ID 0     1
User_ID 1     0
User_ID 2     1
User_ID 3     3
User_ID 4     2
             ..
User_ID 95    1
User_ID 96    1
User_ID 97    3
User_ID 98    0
User_ID 99    0
Name: Some values, Length: 100, dtype: int64

In [55]:
series.values

array([1, 0, 1, 3, 2, 2, 0, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0, 0, 3, 0, 1, 2,
       0, 2, 0, 1, 3, 3, 0, 2, 2, 2, 0, 2, 1, 3, 2, 3, 2, 3, 0, 1, 3, 0,
       0, 0, 2, 3, 0, 0, 0, 1, 2, 0, 0, 2, 0, 1, 0, 3, 1, 2, 2, 0, 2, 0,
       3, 1, 1, 3, 1, 0, 2, 1, 3, 0, 0, 3, 2, 1, 3, 1, 2, 3, 2, 3, 1, 3,
       3, 0, 3, 2, 0, 3, 0, 1, 1, 3, 0, 0])

### Операции
Всё то же самое, что и в `NumPy`.

In [56]:
np.exp(series)

User_ID 0      2.718282
User_ID 1      1.000000
User_ID 2      2.718282
User_ID 3     20.085537
User_ID 4      7.389056
                ...    
User_ID 95     2.718282
User_ID 96     2.718282
User_ID 97    20.085537
User_ID 98     1.000000
User_ID 99     1.000000
Name: Some values, Length: 100, dtype: float64

In [57]:
series + series

User_ID 0     2
User_ID 1     0
User_ID 2     2
User_ID 3     6
User_ID 4     4
             ..
User_ID 95    2
User_ID 96    2
User_ID 97    6
User_ID 98    0
User_ID 99    0
Name: Some values, Length: 100, dtype: int64

In [58]:
series += series
series

User_ID 0     2
User_ID 1     0
User_ID 2     2
User_ID 3     6
User_ID 4     4
             ..
User_ID 95    2
User_ID 96    2
User_ID 97    6
User_ID 98    0
User_ID 99    0
Name: Some values, Length: 100, dtype: int64

### Индексация

In [59]:
series['User_ID 3']

6

In [60]:
series['User_ID 3':'User_ID 95']

User_ID 3     6
User_ID 4     4
User_ID 5     4
User_ID 6     0
User_ID 7     0
             ..
User_ID 91    4
User_ID 92    0
User_ID 93    6
User_ID 94    0
User_ID 95    2
Name: Some values, Length: 93, dtype: int64

### Сохранение в текстовый файл

In [61]:
series.to_csv(to_data_dir('series.csv'))

### Чтение из текстового файла

In [62]:
pd.read_csv(to_data_dir('series.csv'), index_col=0)

,Some values
User_ID 0,2
User_ID 1,0
User_ID 2,2
User_ID 3,6
User_ID 4,4
...,...
User_ID 95,2
User_ID 96,2
User_ID 97,6
User_ID 98,0


По-умолчанию возвращается `pd.DataFrame`. Для того, чтобы по-возможности вернулся `pd.Series`, необходимо определить параметр `squeeze=True`.

In [63]:
pd.read_csv(to_data_dir('series.csv'), index_col=0, squeeze=True)

User_ID 0     2
User_ID 1     0
User_ID 2     2
User_ID 3     6
User_ID 4     4
             ..
User_ID 95    2
User_ID 96    2
User_ID 97    6
User_ID 98    0
User_ID 99    0
Name: Some values, Length: 100, dtype: int64

<div style="text-align: right"><i>Подготовил <a href="https://github.com/andrewsonin">Андрей Сонин</a>